In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import folium
import gmaps
import numpy as np
import requests
import os
# Import API key
from config import gkey
from countryinfo import CountryInfo

In [ ]:
# Stacy's code starts here

## Naturalization - overall data table cleaning

In [ ]:
# Load naturalization table
nat_file = 'naturalization_totals_1907-2018_yb2018.xlsx'
filepath = os.path.join('.', 'Resources', nat_file)

naturalization = pd.read_excel(filepath, header=5)

In [ ]:
# Drop notes data at end of document
naturalization.drop(labels=range(112,118), inplace=True)
naturalization.tail()

In [ ]:
# Rename columns
naturalization.rename(columns={
    'filed': 'Petitions filed',
    'Total': 'Naturalized, total',
    'denied': 'Petitions denied'
}, inplace=True)

# Drop unneeded columns
naturalization = naturalization.drop(labels=['Civilian', 'Military 2', 'Not reported'], axis='columns')

In [ ]:
# Find funny/footnoted years and fix them
for index, row in naturalization.iterrows():
    
    year = naturalization.loc[index, 'Year']
    
    if len(str(year)) > 4:
        year = int(year[0:5])
        naturalization.loc[index, 'Year'] = year
    else:
        pass

# Set year as index
naturalization.set_index(keys=['Year'], inplace=True)

In [ ]:
# Change datatype to int for all columns
for c in naturalization.columns:
    naturalization[c] = naturalization[c].astype('int')

In [ ]:
# Export cleaned naturalization data to df

filename = 'naturalization_filed_denied.csv'
path = os.path.join('.', 'Output_files', filename)
naturalization.to_csv(path)

## Naturalization - by country data cleaning & merging

In [ ]:
# Read in files

filename = 'naturalization_bycountry_2009-2018_table21d_yb2018.xlsx'
filepath = os.path.join('.', 'Resources', filename)
nat_country_20092018 = pd.read_excel(filepath, header=3)

filename = 'naturalization_bycountry_2000-2009_table21d_yb2009.xls'
filepath = os.path.join('.', 'Resources', filename)
nat_country_20002009 = pd.read_excel(filepath, header=3)

filename = 'naturalization_bycountry_1990-1999_table46_yb1999.xlsx'
filepath = os.path.join('.', 'Resources', filename)
nat_country_19901999 = pd.read_excel(filepath, header=4)

nat_country_dfs = [nat_country_20092018, nat_country_20002009, nat_country_19901999]

In [ ]:
# Drop buffer row at start
for df in nat_country_dfs:
    df.drop([0], inplace=True)

In [ ]:
# Drop trailing rows w/ descriptive info
nat_country_19901999.drop(labels=range(204,211), inplace=True)
nat_country_20002009.drop(labels=range(212,220), inplace=True)
nat_country_20092018.drop(labels=range(212,219), inplace=True)

In [ ]:
# Drop region information
nat_country_20002009.drop(index=range(1,10), inplace=True)
nat_country_20092018.drop(index=range(1,10), inplace=True)

In [ ]:
# Standardize column naming
nat_country_19901999.rename(columns={
    'former allegiance': 'Country'
}, inplace=True)
nat_country_20002009.rename(columns={
    'Region and country of birth': 'Country'
}, inplace=True)
nat_country_20092018.rename(columns={
    'Region and country of birth': 'Country'
}, inplace=True)

In [ ]:
# Merge data
nat_country = nat_country_19901999.merge(nat_country_20002009, how='outer', on='Country')
nat_country = nat_country.merge(nat_country_20092018, how='outer', on='Country')
nat_country.head(10)

In [ ]:
# Fill NaNs
nat_country.fillna(0, inplace=True)
nat_country.replace({'X': 0, '-': 0, 'D': 0}, inplace=True)

In [ ]:
# Drop region names from df
nat_country.drop(index=[1, 54, 94, 148, 163, 180, 188], inplace=True)

In [ ]:
# Set index as country names
nat_country.set_index(keys='Country', inplace=True)

# Convert datatype to integer
nat_country.astype('int64').dtypes

In [ ]:
# # Reset max col and row view, to find and fix errant chars
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [ ]:
# Drop redundant 2009 columns
# Note that  this is a quick-and-dirty drop - simply dropping the 'y' col because it has slightly fewer datapoints
# If you compare differences between 2009_x and y, each column reports fewer numbers for around 15 countries
# None of these countries are in our samples, so we're not worrying about them right now
nat_country.drop(columns=['2009_y'], inplace=True)
nat_country.rename(columns={
    '2009_x': '2009'
}, inplace=True)

In [ ]:
# Combine "total data" into "Grand total" row and drop partial rows
all_countries = nat_country.loc['All countries',:]
total_countries = nat_country.loc['Total', :]
total_countries_final = all_countries + total_countries

total_countries_final_df = total_countries_final.to_frame(name='Grand Total')
total_countries_final_df = total_countries_final_df.T

nat_country.drop(index=['All countries'], inplace=True)
nat_country.drop(index=['Total'], inplace=True)

nat_country_df = pd.concat([nat_country, total_countries_final_df])

In [ ]:
# Sort the index so easier to read
nat_country_df.sort_index(axis=0, inplace=True)

In [ ]:
# Export cleaned and merged by-country naturalization data to df

filename = 'naturalization_by_country_merged_1990-2018.csv'
path = os.path.join('.', 'Output_files', filename)
nat_country_df.to_csv(path)

## Asylum seeker demographic data 2018 cleaning

In [ ]:
# Load asylum seeker age/gender/etc table (2018)
asy_2018_file = 'fy2018_table18d_asylum_age_etc.xlsx'
filepath = os.path.join('.', 'Resources', asy_2018_file)

asylum_2018 = pd.read_excel(filepath, header=4)

In [ ]:
asylum_2018.tail(10)

In [ ]:
# Drop notes data at end of document
asylum_2018.drop(labels=range(34,38), inplace=True)
asylum_2018.tail()

In [ ]:
# Split into sex [5:8], rename column
asylum_2018_sex = asylum_2018.iloc[1:4,0:2]
asylum_2018_sex.rename(columns={
    "Characteristic": "Sex"
}, inplace=True)

# reset index
asylum_2018_sex.set_index(keys='Sex', inplace=True)
asylum_2018_sex

# Rename "total" to reflect dataset scope
asylum_2018_sex.rename(columns={
    "Total": "Asylum 2018"
}, inplace=True)
asylum_2018_sex

In [ ]:
# Split out broad age group [27:31]
asylum_2018_broad_age = asylum_2018.iloc[22:27,0:2]
asylum_2018_broad_age_total = asylum_2018_broad_age.drop([22])
asylum_2018_broad_age_total.rename(columns={
    "Characteristic": "Age"
}, inplace=True)
asylum_2018_broad_age_total.set_index(keys='Age', inplace=True)
asylum_2018_broad_age_total

# Rename "total" to reflect dataset scope
asylum_2018_broad_age_total.rename(columns={
    "Total": "Asylum 2018"
}, inplace=True)
asylum_2018_broad_age_total

In [ ]:
# Split out marital status [32:38]
asylum_2018_marital = asylum_2018.iloc[28:34,0:2]
asylum_2018_marital.rename(columns={
    "Characteristic": "Marital Status"
}, inplace=True)
asylum_2018_marital.set_index(keys='Marital Status', inplace=True)
asylum_2018_marital

# Rename "total" to reflect dataset scope
asylum_2018_marital.rename(columns={
    "Total": "Asylum 2018"
}, inplace=True)
asylum_2018_marital

## Asylum seeker demographic data 2009 cleaning

In [ ]:
# Cleaning asylum demographic data from 2009
asy_2009_file = 'fy_2009_table15d_asylum_age_etc.xls'
filepath = os.path.join('.', 'Resources', asy_2009_file)

asylum_2009 = pd.read_excel(filepath, header=5)

In [ ]:
asylum_2009.head(10)

In [ ]:
# Drop notes data at end of document
asylum_2009.drop(labels=range(37,41), inplace=True)
asylum_2009.tail()

In [ ]:
# Split into sex [5:8], rename column
asylum_2009_sex = asylum_2009.iloc[1:4,0:2]
asylum_2009_sex.rename(columns={
    "Characteristic": "Sex"
}, inplace=True)
asylum_2009_sex

# Rename "total" to reflect dataset scope
asylum_2009_sex.rename(columns={
    "Total": "Asylum 2009"
}, inplace=True)

# reset index
asylum_2009_sex.set_index(keys='Sex', inplace=True)
asylum_2009_sex

In [ ]:
# Split out broad age group [32:35]
asylum_2009_broad_age = asylum_2009.iloc[25:29,0:2]
asylum_2009_broad_age.rename(columns={
    "Characteristic": "Age"
}, inplace=True)

asylum_2009_broad_age['Age'] = asylum_2009_broad_age['Age'].str.strip()
asylum_2009_broad_age['Age']

asylum_2009_broad_age.set_index(keys='Age', inplace=True)

asylum_2009_broad_age.index

# Rename "total" to reflect dataset scope
asylum_2009_broad_age.rename(columns={
    "Total": "Asylum 2009"
}, inplace=True)

asylum_2009_broad_age

In [ ]:
# Split out 2009 marital status
asylum_2009_marital = asylum_2009.iloc[31:37,0:2]
asylum_2009_marital.rename(columns={
    "Characteristic": "Marital Status"
}, inplace=True)
asylum_2009_marital.set_index(keys='Marital Status', inplace=True)

# Rename "total" to reflect dataset scope
asylum_2009_marital.rename(columns={
    "Total": "Asylum 2009"
}, inplace=True)

asylum_2009_marital

In [ ]:
# # Clean up inconsistencies between df indices
# asylum_2009_broad_age.rename({'Under 16': '< 16', 
#                                           'Age 16 to 20': '16 - 20', 
#                                           'Age 21 and over': '21+'}, axis='index', inplace=True)

# asylum_2018_broad_age_total.rename({'Under 16 years': '< 16', 
#                                           '16 to 20 years': '16 - 20', 
#                                           '21 years and over': '21+'}, axis='index', inplace=True)

In [ ]:
# # Merge LPR and Asyulym seeker datasets

# # Join asylum datasets together
# lpr_asylum_sex = asylum_2009_sex.merge(asylum_2018_sex, how="inner", left_index=True, right_index=True)
# lpr_asylum_age = asylum_2009_broad_age.merge(asylum_2018_broad_age_total, how="inner", left_index=True, right_index=True)
# lpr_asylum_marital = asylum_2009_marital.merge(asylum_2018_marital, how="inner", left_index=True, right_index=True)

In [ ]:
lpr_asylum_sex

In [ ]:
# Stacy's code ends here

In [ ]:
# Kana's code starts here

# Immigrants by State - overall data table cleaning

In [ ]:
# Read bystate csv data
bystate = pd.read_csv("Resources/By state data.csv")

# Dropna
bystate = bystate.dropna()

# Drop others
bystate.drop(bystate.tail(1).index,inplace=True)

# Change data to integer
bystate.iloc[:,1:20].astype(int)

# Show the dataframe
bystate.head()

In [ ]:
# List of US state abbreviation
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

# Reverse key and value
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [ ]:
# Create an empty dictionary list
dict_list=[]

# For each key and value in dictionary, combine them and add them to a list
for key,value in abbrev_us_state.items():
    dict_list.append((key,value))
    
# Print the list
print (dict_list)

In [ ]:
# Create a dataframe using dictionary list
state_abbrev = pd.DataFrame(dict_list)
state_abbrev.columns = ["Abbrev","State"]

# Show the dataframe
state_abbrev.head()

In [ ]:
# Merge bystate data and state abbreveation dataframe
complete_state_df = pd.merge(bystate, state_abbrev, on = "State")

# Rename columns
complete_state_df = complete_state_df.rename(columns = {"State" : "State Name",
                                                        "Abbrev" : "State"})

# Show the dataframe
complete_state_df.head()

In [ ]:
# Create a new dataframe 
output_state = complete_state_df[["State","2000","2018"]]

# Change data type
convert_dict = {'State': str, 
                '2000': int,
                '2018': int} 

output_state_df = output_state.astype(convert_dict) 

# Display dataframe
output_state.head()

In [ ]:
# Confirm if data type has been changed 
output_state_df.dtypes

In [ ]:
# Output cleaned data as csv
output_state.to_csv("Output_files/Immigrants By State.csv", index = False)

In [ ]:
# Kana's code ends here

In [ ]:
# Satish Start

In [ ]:
# coding=utf-8
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Import API key
from config import gkey
from countryinfo import CountryInfo

In [ ]:
immigration_df=pd.read_csv("Resources/Permanent_Resident_Years.csv")
immigration_df['Number'] = [x.replace(',', '') for x in immigration_df['Number']]

In [ ]:
immigration_df.head()

In [ ]:
immigration_df.Year=pd.to_numeric(immigration_df.Year)
immigration_df.Number=pd.to_numeric(immigration_df.Number)
immigration_plt=immigration_df.plot(kind="line", x="Year", y="Number", grid=True, figsize=(15,10),legend=False,title="Number of Lawful Permanent Resident Status Vs. Years")
max_arrow_y=immigration_df['Number'].max()
max_arrow_x=immigration_df.loc[immigration_df['Number']==max_arrow_y,"Year"].reset_index(drop=True)
plt.annotate(
    f"maximum {max_arrow_x[0],max_arrow_y}", 
    xy=(max_arrow_x[0], max_arrow_y))

min_arrow_y=immigration_df['Number'].min()
min_arrow_x=immigration_df.loc[immigration_df['Number']==min_arrow_y,"Year"].reset_index(drop=True)
plt.annotate(
    f"Minimum {min_arrow_x[0],min_arrow_y}", 
    xy=(min_arrow_x[0], min_arrow_y))

plt.ylabel("Number of Lawful Permanent Resident Status")
plt.xlabel("Timepoint in Years")
plt.tight_layout()
plt.show()

In [ ]:
#Reading CSV
Country_Data_2018=pd.read_csv("Resources/Country_Data_2018.csv")
Country_Data_1999=pd.read_csv("Resources/Country_Data_1999.csv")
Country_Data_2009=pd.read_csv("Resources/Country_Data_2009.csv")

In [ ]:
#Data cleaning,Removing extra column
Country_Data_1999=Country_Data_1999.iloc[:, :-1]

In [ ]:
#Removing Data which has No value
Country_Data_2018_df=Country_Data_2018.dropna()
Country_Data_1999_df=Country_Data_1999.dropna()
Country_Data_2009_df=Country_Data_2009.dropna()

In [ ]:
#Extract First Column so that it can be used for Name as Header
new_header_2018 = Country_Data_2018_df.iloc[0]
new_header_1999 = Country_Data_1999_df.iloc[0]
new_header_2009 = Country_Data_2009_df.iloc[0]

In [ ]:
#Renaming the Header removing first row
Country_Data_2018_df.columns=new_header_2018
Country_Data_2018_df=Country_Data_2018_df[10:]
Country_Data_2018_df.to_csv('Output_files/Country_Data_2018_df.csv')

Country_Data_1999_df.columns=new_header_1999
Country_Data_1999_df=Country_Data_1999_df[1:]
Country_Data_1999_df.to_csv('Output_files/Country_Data_1999_df.csv')

Country_Data_2009_df.columns=new_header_2009
Country_Data_2009_df=Country_Data_2009_df[10:]
Country_Data_2009_df.to_csv('Output_files/Country_Data_2009_df.csv')

In [ ]:
#List of Countries in central America
Central_America_Data=['Mexico', 'Guatemala', 'Honduras', 'Nicaragua', 'El Salvador', 'Costa Rica', 'Panama', 'Belize']
#Getting only records of Central America from main Data Set
Latin_Data_df=Country_Data_2018_df[Country_Data_2018_df['Region and country of birth'].isin(Central_America_Data)]
#Data Cleaning
Latin_Data_df=Latin_Data_df.apply(lambda x: x.str.replace(',',''))

In [ ]:
#Creating new DataFrame for required Data
Latin_Data_summ=[['2014',pd.to_numeric(Latin_Data_df['2014']).sum()],['2015',pd.to_numeric(Latin_Data_df['2015']).sum()],['2017',pd.to_numeric(Latin_Data_df['2017']).sum()],['2018',pd.to_numeric(Latin_Data_df['2018']).sum()]]
Latin_Data_summ_df=pd.DataFrame(Latin_Data_summ, columns = ['Year', 'Count'])

In [ ]:
#Bar Graph showing the Central America and Years
Latin_Data_summ_df.plot.bar(x='Year', y='Count', rot=0,legend=False)
plt.axis('tight')
plt.title("Number of Immigrants Vs Year")
plt.ylabel("Total Number of Immigrants from Central America")
plt.xlabel("Year")
plt.tight_layout()
plt.show()

In [ ]:
#List of Islamic Countries 
Islam_Country_Data=['Afghanistan','Iran','Yemen','Jordan','Saudi Arabia','Sudan','Pakistan','Syria','Oman']
#Data Set till 1999 
Islam_Data_1999_df=Country_Data_1999_df[Country_Data_1999_df['Region and country of birth'].isin(Islam_Country_Data)]
#Data Cleaning
Islam_Data_1999_df=Islam_Data_1999_df.apply(lambda x: x.str.replace(',',''))

In [ ]:
#Data Set form 2000 to 2009 
Islam_Data_2009_df=Country_Data_2009_df[Country_Data_2009_df['Region and country of birth'].isin(Islam_Country_Data)]
#Data Cleaning
Islam_Data_2009_df=Islam_Data_2009_df.apply(lambda x: x.str.replace(',',''))

In [ ]:
#Merging the DataFrame
Merge_Islam_Country=pd.merge(Islam_Data_1999_df,Islam_Data_2009_df,how='outer')
#New DataFrame with reuqired Dataset
Islam_Country_summ=[['1999',pd.to_numeric(Merge_Islam_Country['1999']).sum()],['2000',pd.to_numeric(Merge_Islam_Country['2000']).sum()],['2005',pd.to_numeric(Merge_Islam_Country['2005']).sum()],['20006',pd.to_numeric(Merge_Islam_Country['2006']).sum()]]
Islam_Country_summ=pd.DataFrame(Islam_Country_summ, columns = ['Year', 'Count'])

In [ ]:
#Merging the DataFrame
Merge_Islam_Country=pd.merge(Islam_Data_1999_df,Islam_Data_2009_df,how='outer')
#New DataFrame with reuqired Dataset
Islam_Country_summ=[['1999',pd.to_numeric(Merge_Islam_Country['1999']).sum()],['2000',pd.to_numeric(Merge_Islam_Country['2000']).sum()],['2005',pd.to_numeric(Merge_Islam_Country['2005']).sum()],['20006',pd.to_numeric(Merge_Islam_Country['2006']).sum()]]
Islam_Country_summ=pd.DataFrame(Islam_Country_summ, columns = ['Year', 'Count'])

In [ ]:
#Ployyinh the Graph
Islam_Country_summ.plot.bar(x='Year', y='Count', rot=0,legend=False)
plt.axis('tight')
plt.title("Number of Immigrants Vs Year")
plt.ylabel("Total Number of Immigrants from Arab Countries")
plt.xlabel("Year")
plt.tight_layout()
plt.show()

In [ ]:
Country_Data_1999_2009=pd.merge(Country_Data_1999_df,Country_Data_2009_df,how='outer')

In [ ]:
Country_Data_Merged=pd.merge(Country_Data_1999_2009,Country_Data_2018_df,how='outer')
Country_Data_Merged=Country_Data_Merged.apply(lambda x: x.str.replace(',',''))

In [ ]:
Country_Data_Merged=Country_Data_Merged.dropna()

In [ ]:
Country_Data_Merged.to_csv("Output_files/final.csv")

In [ ]:
Country_Data_Merged=Country_Data_Merged.rename(columns = {'1998 1':'1998'})

In [ ]:
#Country_Data_Merged_clean=Country_Data_Merged.iloc[:,1:(len(Country_Data_Merged.columns)+1)]
header_col=Country_Data_Merged.loc[:,Country_Data_Merged.columns!='Region and country of birth'].columns.tolist()
for col_name in header_col:
    Country_Data_Merged[col_name]=pd.to_numeric(Country_Data_Merged[col_name],errors='coerce').fillna(0).astype(int)

In [ ]:
#Country_Data_Merged.groupby(['Region and country of birth']).sum()
Country_Data_Merged['Total']=Country_Data_Merged.sum(axis=1)

In [ ]:
Country_Data_Merged


In [ ]:
location=[]
#For loop to get the latitude and Longitude
for x in Country_Data_Merged['Region and country of birth']:
    #Get the Lat and Lng from countryinfo package
    try:
        country = CountryInfo(x).info()['latlng']
        location.append(country)
   #If Data is not present in the CountryInfo package then check with Geocode API
    except KeyError:
        base_url="https://maps.googleapis.com/maps/api/geocode/json?address="
        gkey=gkey
        final_url=f"{base_url}{x}&key={gkey}"
        response = requests.get(final_url).json()
        #Getting location and saving results
        result=response['results'][0]['geometry']['location']
        lat=result['lat']
        lng=result['lng']
        latlng=[lat,lng]
        location.append(latlng)

In [ ]:
#Creating new DataFrame for location
location_df=pd.DataFrame(location,columns = ['Lat', 'Lng'])
Country_Data_Merged['Lat']=location_df['Lat'].astype(float)
Country_Data_Merged['Lng']=location_df['Lng'].astype(float)

In [ ]:
#Configuring gmpas
gmaps.configure(api_key=gkey)

fig = gmaps.figure()

#Creating Heat Map
heat_layer = gmaps.heatmap_layer(location_df, weights=Country_Data_Merged['Total'], 
                                 dissipating=False, max_intensity=90000,
                                 point_radius = 1)
#Adding heat maps
fig.add_layer(heat_layer)
fig

In [ ]:
#Saving final merged copy
output_file='Output_files/Country_Data_Merged.csv'
Country_Data_Merged.to_csv(output_file)

In [ ]:
# Satish end

In [ ]:
# Umar's code starts here
#Read in the Excel file and view the headers
Lawful_df = pd.read_excel("./Resources/fy2018_Lawful.xlsx", header=4)
Lawful_df.head()

In [ ]:
#Use the iloc function to locate the point of interest in a data set
Broad_age = Lawful_df.iloc[19:23,:]
Broad_age

In [ ]:
#View the characteristic and Total columns 
Broad_age_df = pd.DataFrame(Broad_age)
Sex = Broad_age_df.iloc[:, 0:2]
Sex

In [ ]:
#Rename your columns 
Cleaned = Sex.rename(columns={"Characteristic": "Age", "Total": "Lawful 2018"})
Index_age = Cleaned.set_index("Age")
Index_age

In [ ]:
#View a single row 
Sex = Lawful_df.iloc[19,1:]
Sex.to_frame(name="LPR 2018")

In [ ]:
#Use the iloc function to locate the point of interest in a data set
Marital_status = Lawful_df.iloc[24:30,:]
Marital_status

In [ ]:
#View the characteristic and Total columns 
New_marital_df = pd.DataFrame(Marital_status)
Specific = New_marital_df.iloc[:, 0:2]
Specific

In [ ]:
#Rename your columns 
Renamed = Specific.rename(columns={"Characteristic": "Marital Status", "Total": "Lawful Permanent Resident 2018"})
Renamed.head()

In [ ]:
#View a single column 
Status = Lawful_df.iloc[24:30,0:2]
Name18 = Status.rename(columns={"Characteristic": "Marital Status", "Total": "Lawful 2018"})
#Set marital status as index for clarity
Name18.set_index("Marital Status", inplace=True)
Name18

In [ ]:
#Read in the Excel file and 
Lawful09_df = pd.read_excel("./Resources/fy2009_Lawful.xls", header=4)
Lawful09_df.head()

In [ ]:
#Use the iloc function to locate the point of interest in a data set
Broad09_age = Lawful09_df.iloc[20:24,:]
Broad09_age

In [ ]:
#View the characteristic and Total columns 
Broad09_age_df = pd.DataFrame(Broad09_age)
Sex09 = Broad09_age_df.iloc[:, 0:2]
Sex09

In [ ]:
#Rename your columns 
Age09 = Sex09.rename(columns={"Characteristic": "Age", "Total": "Lawful Permanent Resident 2009"})
New_index09 = Age09.set_index("Age")
New_index09

In [ ]:
#Use the iloc function to locate the point of interest in a data set
Broad09_age = Lawful09_df.iloc[20:24,:]
Broad09_age

In [ ]:
#View the characteristic and Total columns 
Broad09_age_df = pd.DataFrame(Broad09_age)
Sex09 = Broad09_age_df.iloc[:, 0:2]
Sex09

In [ ]:
#Rename your columns 
Age09 = Sex09.rename(columns={"Characteristic": "Age", "Total": "Lawful Permanent Resident 2009"})
New_index09 = Age09.set_index("Age")
New_index09

In [ ]:
#View a single row 
Sex09 = Lawful09_df.iloc[20,1:]
Sex09 = Sex09.to_frame(name="LPR 2009")

In [ ]:
Sex09

In [ ]:
#Use the iloc function to locate the point of interest in a data set
Marital09_status = Lawful09_df.iloc[26:32,:]
Marital09_status

In [ ]:
#View the characteristic and Total columns 
New09_marital_df = pd.DataFrame(Marital09_status)
Specific09 = New09_marital_df.iloc[:, 0:2]
Specific09

In [ ]:
#Rename your columns 
Renamed09 = Specific09.rename(columns={"Characteristic": "Marital Status", "Total": "Lawful Permanent Resident 2009"})
Renamed09.head()

In [ ]:
#View a single column
Status09 = Lawful09_df.iloc[26:32,0:2]
Name09 = Status09.rename(columns={"Characteristic": "Marital Status", "Total": "Lawful 2009"})
#Set marital status as index for clarity
Name09.set_index("Marital Status", inplace=True)
Name09

#Umar Code ends here

### Merge LPR and asylum seeker datasets

In [ ]:
# Stacy code for merging datasets starts here

In [ ]:
# Clean up inconsistencies between df indices and types
asylum_2009_broad_age.rename({'Under 16': '< 16', 
                                          'Age 16 to 20': '16 - 20', 
                                          'Age 21 and over': '21+'}, axis='index', inplace=True)

asylum_2018_broad_age_total.rename({'Under 16 years': '< 16', 
                                          '16 to 20 years': '16 - 20', 
                                          '21 years and over': '21+'}, axis='index', inplace=True)

sex_18 = Sex.to_frame(name="Lawful 2018")

In [ ]:
# Merge LPR and Asyulym seeker datasets

# Join asylum datasets together
lpr_asylum_sex = asylum_2009_sex.merge(asylum_2018_sex, how="inner", left_index=True, right_index=True)
lpr_asylum_age = asylum_2009_broad_age.merge(asylum_2018_broad_age_total, how="inner", left_index=True, right_index=True)
lpr_asylum_marital = asylum_2009_marital.merge(asylum_2018_marital, how="inner", left_index=True, right_index=True)

# Join LPR datasets together 
lpr_age = New_index09.merge(Index_age, how="inner", left_index=True, right_index=True)
lpr_sex = Sex09.merge(sex_18, how="inner", left_index=True, right_index=True)
lpr_marital = Name09.merge(Name18, how="inner", left_index=True, right_index=True)

In [ ]:
# Clean up inconsistencies on index and column names

for c in lpr_age.index:
    lpr_age.rename(index={
    c: str(c).strip()
}, inplace=True)
    
lpr_age.rename({'Under 16 years': '< 16', 
               '16 to 20 years': '16 - 20', 
               '21 years and over': '21+'}, axis='index', inplace=True)

lpr_age.rename({'Lawful Permanent Resident 2009': 'Lawful 2009'})

lpr_sex.rename(columns={'LPR 2009': 'Lawful 2009'}, inplace=True)

In [ ]:
lpr_asylum_sex = lpr_asylum_sex.merge(lpr_sex, how="inner", left_index=True, right_index=True)
lpr_asylum_age = lpr_asylum_age.merge(lpr_age, how="inner", left_index=True, right_index=True)
lpr_asylum_marital = lpr_asylum_marital.merge(lpr_marital, how="inner", left_index=True, right_index=True)

In [ ]:
lpr_asylum_sex

In [ ]:
# Export cleaned data to Outputs folder

filename = 'la_sex.csv'
path = os.path.join('.', 'Output_files', filename)
lpr_asylum_sex.to_csv(path)

filename = 'la_age.csv'
path = os.path.join('.', 'Output_files', filename)
lpr_asylum_age.to_csv(path)

filename = 'la_marital.csv'
path = os.path.join('.', 'Output_files', filename)
lpr_asylum_marital.to_csv(path)

In [ ]:
# Stacy code for merging datasets ends here